In [4]:
from dateutil.easter import easter
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

# Deine CSV-Datei laden
csv_path = 'anzahlEinAus.csv'
df = pd.read_csv(csv_path)

# Konvertiere 'Datum' in ein DateTime-Objekt und füge 'Jahr' hinzu
df['Datum'] = pd.to_datetime(df['Datum'])
df['Jahr'] = df['Datum'].dt.year

# Berechne die Differenz zwischen Eintritten und Austritten
df['Differenz'] = df['Anzahl Eintritte'] - df['Anzahl Austritte']

# Eindeutige Liste der Jahre für das Dropdown-Menü
jahre = df['Jahr'].unique()
jahre.sort()

# Initialisiere die Dash-App
app = Dash(__name__)

# App-Layout
app.layout = html.Div([
    dcc.Dropdown(
        id='jahr-dropdown',
        options=[{'label': jahr, 'value': jahr} for jahr in jahre],
        value=jahre[0]  # Standardwert ist das erste Jahr
    ),
    dcc.Graph(id='graph'),
])

@app.callback(
    Output('graph', 'figure'),
    [Input('jahr-dropdown', 'value')]
)
def update_graph(selected_year):
    filtered_df = df[df['Jahr'] == selected_year]
    fig = go.Figure()

    # Hinzufügen der Balken und Linien für die Daten
    fig.add_trace(go.Bar(x=filtered_df['Datum'].dt.strftime('%Y-%m-%d'), y=filtered_df['Anzahl Eintritte'], name='Eintritte', marker_color='coral'))
    fig.add_trace(go.Bar(x=filtered_df['Datum'].dt.strftime('%Y-%m-%d'), y=-filtered_df['Anzahl Austritte'], name='Austritte', marker_color='cornflowerblue'))
    fig.add_trace(go.Scatter(x=filtered_df['Datum'].dt.strftime('%Y-%m-%d'), y=filtered_df['Differenz'], mode='lines+markers', name='Differenz', line=dict(color='green')))

    # Feiertage markieren und beschriften
    annotations = []
    holidays = {
        "Ostern": {"date": easter(selected_year), "color": "orange"},
        "Weihnachten": {"date": pd.Timestamp(year=selected_year, month=12, day=24), "color": "blue"},
        "Neujahr": {"date": pd.Timestamp(year=selected_year+1, month=1, day=1), "color": "blue"},
        "Basler Fasnacht": {"date": easter(selected_year) - pd.Timedelta(days=46) + pd.Timedelta(days=4), "color": "green"}
    }

    annotation_x_positions = [0.5, 0.35, 0.65, 0.95]  # Definiere feste x-Positionen für jede Annotation

    for i, (holiday, info) in enumerate(holidays.items()):
        start_date = info["date"]
        end_date = start_date + pd.Timedelta(days=1) if holiday != "Basler Fasnacht" else start_date + pd.Timedelta(days=3)
        fig.add_vrect(x0=start_date.strftime('%Y-%m-%d'), x1=end_date.strftime('%Y-%m-%d'), fillcolor=info["color"], opacity=0.5, layer="below", line_width=0)
        
        # Füge die Annotation zur Liste hinzu
        annotations.append(dict(
            xref="paper", yref="paper",
            x=annotation_x_positions[i], y=1.05,
            text=f"{holiday}: {start_date.strftime('%Y-%m-%d')}",
            showarrow=False,
            font=dict(size=12, color=info["color"]),
            align="center",
            bgcolor="rgba(255,255,255,0.8)"
        ))
    # Wochenenden (Samstage und Sonntage) markieren
    start_date = filtered_df['Datum'].min()
    end_date = filtered_df['Datum'].max()
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 5:  # Samstag
            # Füge die Markierung für Samstag hinzu
            fig.add_vrect(
                x0=current_date.strftime('%Y-%m-%d'), 
                x1=(current_date + pd.Timedelta(days=1)).strftime('%Y-%m-%d'),
                fillcolor="grey", opacity=0.2, layer="below", line_width=0
            )
        elif current_date.weekday() == 6:  # Sonntag
            # Da Sonntag bereits am x0 beginnt, fügen wir keine zusätzliche Markierung hinzu,
            # um eine Doppelmarkierung oder Überlappung zu vermeiden.
            pass
        current_date += pd.Timedelta(days=1)

        # Sommerferien für Basel-Stadt hinzufügen
    sommerferien = {
        2019: {"start": "2019-06-29", "end": "2019-08-10"},
        2020: {"start": "2020-06-27", "end": "2020-08-08"},
        2021: {"start": "2021-07-03", "end": "2021-08-14"},
        2022: {"start": "2022-07-02", "end": "2022-08-13"},
        2023: {"start": "2023-07-01", "end": "2023-08-12"},
        2024: {"start": "2024-06-29", "end": "2024-08-10"},
    }

    """

    # Summen für das ausgewählte Jahr berechnen
    summe_eintritte = filtered_df['Anzahl Eintritte'].sum()
    summe_austritte = filtered_df['Anzahl Austritte'].sum()

    # Annotations für die Summen hinzufügen
    fig.add_annotation(xref="paper", yref="paper", x=1.1, y=1,
                       text=f"Summe Eintritte {selected_year}: {summe_eintritte}<br>Summe Austritte {selected_year}: {summe_austritte}",
                       showarrow=False, font=dict(size=12),
                       align="left")

    if selected_year in sommerferien:
        ferien_info = sommerferien[selected_year]
        fig.add_vrect(
            x0=ferien_info["start"], x1=ferien_info["end"],
            fillcolor="lightblue", opacity=0.5, layer="below", line_width=0,
            annotation_text="Sommerferien", annotation_position="top left"
        )
    """


    # x-Achse auf den Bereich der vorhandenen Daten beschränken
    fig.update_xaxes(range=[start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')])
    # Update das Layout mit den Annotations
    fig.update_layout(
        title="Eintritte und Austritte nach Jahr", 
        xaxis_title="Datum", 
        yaxis_title="Anzahl", 
        barmode='relative', 
        xaxis=dict(type='date'),
        height=600, 
        width=1500,
        margin=dict(l=100, r=100, t=100, b=100),
        legend=dict(
            y=1.1, 
            x=0.5, 
            xanchor='center', 
            yanchor='top', 
            orientation='h'
        ),
        yaxis=dict(
            range=[-50, 50]
        ),
        annotations=[
            #dict(xref="paper", yref="paper", x=0.15, y=1.05, text=f"Basler Fasnacht: {(easter(selected_year) - pd.Timedelta(days=46) + pd.Timedelta(days=4)).strftime('%Y-%m-%d')}", showarrow=False, font=dict(size=12, color="green")),
            dict(xref="paper", yref="paper", x=0.5, y=1.05, text=f"Ostern: {easter(selected_year).strftime('%Y-%m-%d')}", showarrow=False, font=dict(size=12, color="orange")),
            #dict(xref="paper", yref="paper", x=0.7, y=-0.2, text=f"Weihnachten: {pd.Timestamp(year=selected_year, month=12, day=24).strftime('%Y-%m-%d')}", showarrow=False, font=dict(size=12, color="blue")),
            #dict(xref="paper", yref="paper", x=0.9, y=-0.2, text=f"Neujahr: {pd.Timestamp(year=selected_year+1, month=1, day=1).strftime('%Y-%m-%d')}", showarrow=False, font=dict(size=12, color="blue")),
        ]
    )

    return fig


# Führe die App aus
if __name__ == '__main__':
    app.run_server(debug=True, port=8061)
    app.run_server(debug=True)


AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.